In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [10]:
from math import sqrt


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [34]:
df = pd.DataFrame(columns=["shoulder_distance", "feet_distance"])

df.columns

,shoulder_distance,feet_distance


In [63]:
cap = cv2.VideoCapture("../data/squat/squat_right_2.mp4")

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        shoulder_width = feet_width = None

        # Calculate and compare the distance of 2 ankle and shoulder width
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Calculate shoulder width
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            shoulder_width = calculate_distance(left_shoulder, right_shoulder)

            # Calculate 2-feet width
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            feet_width = calculate_distance(left_ankle, right_ankle)

            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            # Display feet distance
            cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display shoulder distance
            cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            
        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow("CV2", image)

        # Pressed key for action
        k = cv2.waitKey(1) & 0xFF

        # Press C to add to df
        if k == ord('c'): 
            df = df.append({ "shoulder_distance": shoulder_width, "feet_distance": feet_width }, ignore_index=True)

        # Press q to stop
        elif k == ord("q"):
            break
        else: continue

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)

In [39]:
df.head(10)

,shoulder_distance,feet_distance
0,0.078844,0.086320
1,0.079312,0.086597
2,0.079516,0.086532
3,0.079603,0.086277
4,0.079598,0.086257
5,0.079534,0.086387
6,0.079307,0.086514
7,0.079485,0.086594
8,0.079411,0.086708
9,0.079535,0.085932


In [40]:
df["ratio"] = df["feet_distance"] / df["shoulder_distance"]

In [41]:
df

,shoulder_distance,feet_distance,ratio
0,0.078844,0.086320,1.094831
1,0.079312,0.086597,1.091857
2,0.079516,0.086532,1.088229
3,0.079603,0.086277,1.083839
4,0.079598,0.086257,1.083657
...,...,...,...
356,0.096024,0.109927,1.144781
357,0.097100,0.110258,1.135514
358,0.096212,0.110275,1.146166
359,0.093908,0.110023,1.171604


In [42]:
df["ratio"].mean(axis=0)

1.3505960467988047

In [45]:
df["ratio"].idxmin()

45

In [54]:
import math

df["ratio_rounded_up"] = [round(row, 1) for row in df["ratio"]]

In [55]:
df

,shoulder_distance,feet_distance,ratio,ratio_rounded_up
0,0.078844,0.086320,1.094831,1.1
1,0.079312,0.086597,1.091857,1.1
2,0.079516,0.086532,1.088229,1.1
3,0.079603,0.086277,1.083839,1.1
4,0.079598,0.086257,1.083657,1.1
...,...,...,...,...
356,0.096024,0.109927,1.144781,1.1
357,0.097100,0.110258,1.135514,1.1
358,0.096212,0.110275,1.146166,1.1
359,0.093908,0.110023,1.171604,1.2


In [56]:
df["ratio_rounded_up"].mean(axis=0)


1.3537396121883654

In [60]:
df["ratio_rounded_up"].idxmax()

72

In [62]:
df[71:73]

,shoulder_distance,feet_distance,ratio,ratio_rounded_up
71,0.079309,0.085370,1.076416,1.1
72,0.074603,0.126595,1.696911,1.7


In [65]:
df["ratio_rounded_up"].max(), df["ratio_rounded_up"].min()

(1.7, 1.0)